# **C.O.L.L.A.B.**
<hr style="border:2px solid black">

## Goal: 
* 
* 
<hr style="border:2px solid black">

# Imports

In [1]:
import pandas as pd
import numpy as np


<hr style="border:2px solid black">

# **Acquire**
Plan --> **Acquire** --> Prepare --> Explore --> Model --> Deliver

* 

In [2]:
# Acqure kaggle data
col = pd.read_csv('cost_of_living_us.csv')
dem = pd.read_csv('county_demographics.csv')

# Acquire Census Data
family_income = pd.read_csv('B19126.csv')
family_count = pd.read_csv('DP02.csv')

<hr style="border:2px solid black">

# **Prepare**

**Prepare Actions:**
* To better estimate family income based on family composition, use the estimates of family count by family type(Married, Single Male with kids, Single Male without kids, Single Female with kids, Single Female without kids) and the estimates of income by family type. e.g. (female_with_kids (income) * female_with_kids (count) + (male_with_kids (income) * male_with_kids (count)))/2
* Merge transformed census dataframe (family) on to the cost_of_living dataframe (col) on county.
* Adjust for inflation 2020 to 2022

In [3]:
male = family_count[6:8]
female = family_count[10:12]
family_count = pd.concat([female, male], ignore_index=True, axis = 0)
columns = family_count.columns[2::2]
family_count.drop(columns, axis = 1, inplace = True)
family_count = family_count.T.rename(columns = {0:'fem_total(count)', 1:'fem_with_kid(count)', 2:'male_total(count)', 3:'male_with_kid(count)'})
family_count = family_count.drop(family_count.index[0])

In [4]:
family_count = family_count.fillna('0')
family_count['fem_total(count)'] = family_count['fem_total(count)'].str.replace(',', '').astype(int)
family_count['fem_with_kid(count)'] = family_count['fem_with_kid(count)'].str.replace(',', '').astype(int)
family_count['male_total(count)'] = family_count['male_total(count)'].str.replace(',', '').astype(int)
family_count['male_with_kid(count)'] = family_count['male_with_kid(count)'].str.replace(',', '').astype(int)

# transform strings into floats
family_count['fem_no_kid(count)'] = family_count['fem_total(count)'] - family_count['fem_with_kid(count)']
family_count['male_no_kid(count)'] = family_count['male_total(count)'] - family_count['male_with_kid(count)']

In [5]:
tot = family_income[4:6]
male = family_income[10:12]
female = family_income[14:16]
family_income = pd.concat([tot, female, male], ignore_index=True, axis = 0)
family_income = family_income.T.rename(columns = {0:'mar_with_kid', 1:'mar_no_kid', 2:'fem_with_kid(income)', 
                                                  3:'fem_no_kid(income)', 4:'male_with_kid(income)', 5:'male_no_kid(income)'})
family_income = family_income.drop(family_income.index[0])

In [6]:
family_income['mar_with_kid'] = family_income['mar_with_kid'].str.replace('+', '')
family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')
family_income['male_with_kid(income)'] = family_income['male_no_kid(income)'].str.replace('+', '').replace('-', '')

In [7]:
family_income['mar_with_kid'] = family_income['mar_with_kid'].replace('', '0').str.replace(',', '').astype(int)
family_income['mar_no_kid'] = family_income['mar_no_kid'].replace('', '0').str.replace(',', '').astype(int)
family_income['fem_no_kid(income)'] = family_income['fem_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
family_income['fem_with_kid(income)'] = family_income['fem_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
family_income['male_no_kid(income)'] = family_income['male_no_kid(income)'].replace('', '0').str.replace(',', '').astype(int)
family_income['male_with_kid(income)'] = family_income['male_with_kid(income)'].replace('', '0').str.replace(',', '').astype(int)

In [8]:
family_income.dtypes

mar_with_kid             int64
mar_no_kid               int64
fem_with_kid(income)     int64
fem_no_kid(income)       int64
male_with_kid(income)    int64
male_no_kid(income)      int64
dtype: object

In [9]:
family = pd.concat([family_income, family_count], axis = 1)

In [10]:
family

,mar_with_kid,mar_no_kid,fem_with_kid(income),fem_no_kid(income),male_with_kid(income),male_no_kid(income),fem_total(count),fem_with_kid(count),male_total(count),male_with_kid(count),fem_no_kid(count),male_no_kid(count)
"Baldwin County, Alabama!!Estimate",106141,91016,28433,51519,93322,93322,22981,4249,13426,950,18732,12476
"Calhoun County, Alabama!!Estimate",69568,86169,29121,48318,74722,74722,15956,3779,9599,318,12177,9281
"Cullman County, Alabama!!Estimate",85096,88386,21387,36857,91032,91032,8584,1402,5554,500,7182,5054
"DeKalb County, Alabama!!Estimate",57004,75605,28830,47992,144039,144039,8026,1343,4541,362,6683,4179
"Elmore County, Alabama!!Estimate",95917,101406,39276,61279,72352,72352,9466,2283,4373,727,7183,3646
...,...,...,...,...,...,...,...,...,...,...,...,...
"Ponce Municipio, Puerto Rico!!Estimate",37900,27092,3513,20286,31092,31092,0,0,0,0,0,0
"San Juan Municipio, Puerto Rico!!Estimate",76165,52769,8564,28066,31557,31557,0,0,0,0,0,0
"Toa Alta Municipio, Puerto Rico!!Estimate",48214,64840,19045,32256,82121,82121,0,0,0,0,0,0
"Toa Baja Municipio, Puerto Rico!!Estimate",65154,43236,11675,35407,29614,29614,0,0,0,0,0,0


<hr style="border:2px solid black">

# **Explore**
Plan --> Acquire --> Prepare --> **Explore** --> Model --> Deliver

<hr style="border:2px solid black">

# **Modeling**
Plan --> Acquire --> Prepare --> Explore --> **Model** --> Deliver


<hr style="border:2px solid black">

# **Comparing Models**


<hr style="border:2px solid black">

## **Best Model Evaluation on Test**

### <u>Modeling Summary:</u>

>* 

<hr style="border:2px solid black">

# **Conclusions**


### <u>Modeling:</u>

>**<u>:</u>**
>* 

### <u>Recommendations:</u>
>* 

### <u>Next Steps:</u>
>* 